In [ ]:
%reload_ext autoreload
%autoreload 2

import pandas as pd

from src.acnets.pipeline import Parcellation, ConnectivityExtractor, ConnectivityAggregator
from sklearn.preprocessing import LabelEncoder
from typing_extensions import deprecated


In [ ]:
# Xy
subjects = Parcellation(atlas_name='difumo_64_2mm').fit_transform(None).coords['subject'].values
X = subjects.reshape(-1,1)                                  # subjects ids, shape: (n_subjects, 1)

y_encoder = LabelEncoder()
y = y_encoder.fit_transform([s[:4] for s in subjects])      # labels (AVGP=1 or NVGP=1), shape: (n_subjects,)


In [ ]:
import numpy as np

p = Parcellation('dosenbach2010')
p_res = p.fit_transform(X)

c = ConnectivityExtractor('correlation')
c_res = c.fit_transform(p_res)

a = ConnectivityAggregator('network')
a_res = a.fit_transform(c_res)
a_res

In [ ]:
# Flatten connectivity matrix

@deprecated('Use full connectivity matrix instead')
def flatten(ds, mask_k=0):

    ds = ds.copy()
    import numpy as np
    node_type = ds['connectivity'].dims[-1][:-4]
    c = ds['connectivity'].values
    c[..., *np.tril_indices(c.shape[1], k=mask_k)] = np.nan
    ds['connectivity'].values = c
    c = ds['connectivity'].to_dataframe().dropna().reset_index().rename(columns={
        'connectivity': 'partial correlation'})
    c['connectivity'] = c[f'{node_type}_src'] + ' - ' + c[f'{node_type}_dst']
    c = c.pivot(index='subject', columns='connectivity', values='partial correlation')
    return c